In [1]:
import pygame
import time
from PIL import Image, ImageSequence
import numpy as np
import cv2
import mediapipe as mp
import os
import sounddevice as sd
from scipy.signal import butter, lfilter

pygame 2.6.1 (SDL 2.28.4, Python 3.10.16)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
# Inisialisasi pygame
pygame.init()
pygame.mixer.init()

# Fungsi untuk memainkan suara (tetap sama)
def play_sound(file_path):
    try:
        sound = pygame.mixer.Sound(file_path)
        sound.play()
        while pygame.mixer.get_busy():
            time.sleep(0.1)
    except pygame.error as e:
        print(f"Error playing sound '{file_path}': {e}")

# Fungsi load_gif_frames dan paste_character (tetap sama)
def load_gif_frames(path):
    try:
        gif = Image.open(path)
        frames = []
        for frame in ImageSequence.Iterator(gif):
            frame = frame.convert("RGBA").resize((60, 60))
            frames.append(np.array(frame))
        return frames
    except FileNotFoundError:
        print(f"Error: GIF file not found at '{path}'")
        return []

def paste_character(background, character, x, y):
    h, w, _ = character.shape
    bg_section = background[y:y+h, x:x+w].copy()
    alpha_char = character[:, :, 3] / 255.0
    alpha_bg = 1.0 - alpha_char
    for c in range(3):
        bg_section[:, :, c] = (alpha_char * character[:, :, c] +
                               alpha_bg * bg_section[:, :, c])
    background[y:y+h, x:x+w] = bg_section
    return background

# Fungsi bantu calculate_sum dan is_hand_moving (tetap sama)
def calculate_sum(landmark_list):
    if landmark_list:
        return sum(landmark_list[i].x * 480 for i in [11, 12, 13, 14, 15, 16, 23, 24])
    return 0

def is_hand_moving(landmarks, prev_pos, threshold=15):
    if landmarks and len(landmarks) > 16:
        if prev_pos is None:
            return False, [landmarks[15].x, landmarks[16].x]
        left_movement = abs(landmarks[15].x - prev_pos[0]) > threshold / 640
        right_movement = abs(landmarks[16].x - prev_pos[1]) > threshold / 640
        return left_movement or right_movement, [landmarks[15].x, landmarks[16].x]
    return False, None

def is_visible(landmark_list):
    return (len(landmark_list) > 24 and
            landmark_list[11].visibility > 0.7 and
            landmark_list[12].visibility > 0.7)

# Implementasi Band-Pass Filter
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

def get_user_voice_volume(threshold=0.05, lowcut=300.0, highcut=3000.0, fs=44100, order=5):
    try:
        duration = 0.5
        audio = sd.rec(int(duration * fs), samplerate=fs, channels=1, dtype='float32')
        sd.wait()
        filtered_audio = butter_bandpass_filter(audio[:, 0], lowcut, highcut, fs, order=order)
        return np.linalg.norm(filtered_audio) / len(filtered_audio) > threshold
    except Exception as e:
        print(f"Error recording audio: {e}")
        return False

def game_loop():
    # Inisialisasi webcam
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Tidak dapat mengakses webcam")
        return

    # Setup MediaPipe pose
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose()
    drawing = mp.solutions.drawing_utils

    # Load aset
    gif_path = os.path.join(os.getcwd(), 'mario-icegif-6.gif')
    mario_frames = load_gif_frames(gif_path)
    mario_frame_index = 0
    mario_total_frames = len(mario_frames)

    try:
        im1 = cv2.imread('im1.png')
        im2 = cv2.imread('im2.png')
    except FileNotFoundError as e:
        print(f"Error loading image: {e}")
        im1 = np.zeros((480, 640, 3), dtype=np.uint8)
        im2 = np.zeros((480, 640, 3), dtype=np.uint8)

    current_window = im1

    # Variabel game
    is_init = False
    is_cinit = False
    start_time, end_time = 0, 0
    c_start_time, c_end_time = 0, 0
    duration = 0
    c_position = 0
    winner = 0
    is_alive = 1
    user_sum = 0
    temp_sum = 0
    prev_hand_pos = None
    threshold_move = 150
    in_frame = 0
    in_frame_check = False
    threshold_dist = 180

    # Waktu permainan
    game_start_time = time.time()
    game_duration = np.random.randint(10, 31)

    # Untuk menyimpan video
    should_save_video = False
    video_writer = None
    output_filename = "game_result.avi"
    fps = 20
    frame_size = (800, 400 + 480)

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Gagal mengambil frame dari webcam")
            break

        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(rgb)
        frame = cv2.blur(frame, (5, 5))

        if results.pose_landmarks:
            drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        if not in_frame_check:
            try:
                if is_visible(results.pose_landmarks.landmark):
                    in_frame = 1
                    in_frame_check = True
            except:
                pass

        if in_frame == 1:
            if not is_init:
                play_sound('greenLight.mp3')
                current_window = im1
                start_time = time.time()
                end_time = start_time
                duration = np.random.randint(2, 5)
                is_init = True

            if (end_time - start_time) <= duration:
                moved = False
                voiced = False
                if results.pose_landmarks:
                    hand_moved_green, prev_hand_pos = is_hand_moving(results.pose_landmarks.landmark, prev_hand_pos)
                    moved = hand_moved_green

                voice_detected_green = get_user_voice_volume()
                voiced = voice_detected_green

                if moved or voiced:
                    try:
                        m = results.pose_landmarks.landmark[11].y * 640 - results.pose_landmarks.landmark[23].y * 640
                        if m < threshold_dist:
                            c_position += 1
                        print("Progress (karena gerak/suara):", c_position)
                    except:
                        pass
                else:
                    print("Progress (tidak ada aksi):", c_position)

                end_time = time.time()
            else:
                if c_position >= 100:
                    winner = 1
                else:
                    if not is_cinit:
                        is_cinit = True
                        cv2.putText(frame, "Bersiap untuk Red Light...", (200, 200), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), 4)
                        cv2.imshow("Main Window", frame)
                        cv2.waitKey(1000)
                        c_start_time = time.time()
                        c_end_time = c_start_time
                        current_window = im2
                        play_sound('redLight.mp3')
                        if results.pose_landmarks:
                            user_sum = calculate_sum(results.pose_landmarks.landmark)

                    if (c_end_time - c_start_time) <= 3 and results.pose_landmarks:
                        temp_sum = calculate_sum(results.pose_landmarks.landmark)
                        c_end_time = time.time()
                        hand_moved, _ = is_hand_moving(results.pose_landmarks.landmark, prev_hand_pos)
                        voice_detected = get_user_voice_volume()
                        if abs(temp_sum - user_sum) > threshold_move or hand_moved or voice_detected:
                            print("DEAD: Bergerak atau bersuara saat lampu merah!")
                            is_alive = 0
                        else:
                            is_init = False
                            is_cinit = False

            if mario_frames:
                mario_h, mario_w, _ = mario_frames[0].shape
                mario_x = 55 + 6 * c_position
                mario_y = 280
                if mario_x + mario_w > current_window.shape[1]:
                    mario_x = current_window.shape[1] - mario_w
                mario_frame = mario_frames[mario_frame_index % mario_total_frames]
                current_window = paste_character(current_window, mario_frame, mario_x, mario_y)
                mario_frame_index += 1

            # Cek waktu permainan dan tampilkan layar
            current_game_time = time.time() - game_start_time
            if current_game_time >= game_duration and winner == 0:
                is_alive = 0
                print("Waktu habis!")

            if im1 is not None and current_window is not None:
                resized_frame = cv2.resize(frame, (800, 400))
                main_window = np.concatenate((resized_frame, current_window), axis=0)
                remaining_time = max(0, int(game_duration - current_game_time))
                cv2.putText(resized_frame, f"Waktu Tersisa: {remaining_time} detik", (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 3)
                cv2.imshow("Main Window", main_window)
                if should_save_video and video_writer is not None:
                    video_writer.write(main_window)
            else:
                temp_frame = cv2.resize(frame, (800, 680))
                cv2.imshow("Main Window", temp_frame)
                if should_save_video and video_writer is not None:
                    black_canvas = np.zeros((480, 800, 3), dtype=np.uint8)
                    combined_frame = np.concatenate((cv2.resize(frame, (800, 400)), black_canvas), axis=0)
                    video_writer.write(combined_frame)

        else:
            current_game_time = time.time() - game_start_time
            cv2.putText(frame, "Silakan pastikan tubuh bagian atas terlihat di kamera", (20, 200), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 4)
            remaining_time = max(0, int(game_duration - current_game_time))
            cv2.putText(frame, f"Waktu Tersisa: {remaining_time} detik", (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 3)
            cv2.imshow("Main Window", frame)
            if should_save_video and video_writer is not None:
                black_canvas = np.zeros((480, 800, 3), dtype=np.uint8)
                combined_frame = np.concatenate((cv2.resize(frame, (800, 400)), black_canvas), axis=0)
                video_writer.write(combined_frame)

        key = cv2.waitKey(1)
        if key == 27 or is_alive == 0 or winner == 1:
            break
        elif key == ord('s') and not should_save_video:
            should_save_video = True
            fourcc = cv2.VideoWriter_fourcc(*'XVID')
            video_writer = cv2.VideoWriter(output_filename, fourcc, fps, (800, 400 + 480))
            print("Mulai merekam video...")
        elif key == ord('e') and should_save_video:
            should_save_video = False
            if video_writer is not None:
                video_writer.release()
                video_writer = None
            print(f"Video disimpan sebagai {output_filename}")

    cap.release()
    cv2.destroyAllWindows()

    # Hasil akhir
    print("\n--- Hasil Akhir ---")
    result_text = ""
    if is_alive == 0:
        if winner == 1:
            result_text = "Selamat! Kamu Menang."
            print(result_text)
        elif time.time() - game_start_time >= game_duration:
            result_text = "Kamu Gagal: Waktu habis."
            print(result_text)
        else:
            result_text = "Kamu Gagal: Bergerak/bersuara saat lampu merah."
            print(result_text)
    elif winner == 1:
        result_text = "Selamat! Kamu Menang."
        print(result_text)
    else:
        result_text = "Permainan berakhir."
        print(result_text)

    # Tanyakan penyimpanan video dan tampilkan pesan di layar OpenCV
    save_prompt_text = "Simpan video hasil permainan? (y/n): "
    save_prompt = input(save_prompt_text).lower()
    video_saved_message = ""
    if save_prompt == 'y':
        if video_writer is not None:
            video_writer.release()
            video_saved_message = f"Video disimpan sebagai {output_filename}"
            print(video_saved_message)
        elif not should_save_video:
            video_saved_message = "Perekaman tidak diaktifkan, video tidak disimpan."
            print(video_saved_message)
    else:
        video_saved_message = "Video tidak disimpan."
        print(video_saved_message)

    # Tampilkan pesan hasil dan penyimpanan video menggunakan OpenCV
    message_canvas = np.zeros((250, 700, 3), dtype=np.uint8) + 255 # Canvas putih
    y_offset = 50
    cv2.putText(message_canvas, "--- Hasil Permainan ---", (50, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)
    y_offset += 50
    cv2.putText(message_canvas, result_text, (50, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
    y_offset += 50
    cv2.putText(message_canvas, save_prompt_text + save_prompt, (50, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (100, 100, 100), 2)
    y_offset += 50
    cv2.putText(message_canvas, video_saved_message, (50, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 128, 0), 2)

    cv2.imshow("Hasil Permainan", message_canvas)
    cv2.waitKey(0)
    cv2.destroyWindow("Hasil Permainan")
    
if __name__ == '__main__':
    game_loop()

Progress (tidak ada aksi): 0
Progress (karena gerak/suara): 1
Progress (karena gerak/suara): 2
Progress (karena gerak/suara): 3
Progress (karena gerak/suara): 4
DEAD: Bergerak atau bersuara saat lampu merah!

--- Hasil Akhir ---
Kamu Gagal: Bergerak/bersuara saat lampu merah.
Perekaman tidak diaktifkan, video tidak disimpan.


error: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window_w32.cpp:1261: error: (-27:Null pointer) NULL window: 'Hasil Permainan' in function 'cvDestroyWindow'
